In [82]:
import pandas as pd
import numpy as np
import time
import re
import ast
import nltk
from nltk.corpus import stopwords

start_time = time.time()

reviews = pd.read_pickle("florida_features_with_text.p")
reviews = reviews.iloc[0:5]
reviews2 = reviews.copy()
lex = pd.read_csv("Trafficking_Lexicon.csv").dropna().rename(columns={'Human Trafficking (weight 0-1-2)':'HT_Weight',
                                                                     "Test Weight (0-1-2)":"Sex_Weight"})
sw = set(stopwords.words('english'))

In [87]:
'so' in sw

True

In [10]:
ht_dict = pd.Series(lex.HT_Weight.values, index=lex.TERM)
sex_dict = pd.Series(lex.Sex_Weight.values, index=lex.TERM)

ht_dict = ht_dict.reset_index()
ht_dict['TERM'] = ht_dict['TERM'].apply(lambda x: x.lower())
ht_dict.index = ht_dict["TERM"]
ht_dict = ht_dict.drop("TERM", axis=1)
ht_dict = ht_dict.to_dict()[0]

sex_dict = sex_dict.reset_index()
sex_dict['TERM'] = sex_dict['TERM'].apply(lambda x: x.lower())
sex_dict.index = sex_dict["TERM"]
sex_dict = sex_dict.drop("TERM", axis=1)
sex_dict = sex_dict.to_dict()[0]

In [65]:
def lex_score_real(row):
    ht_0 = []
    ht_1 = []
    ht_2 = []
    sex_0 = []
    sex_1 = []
    sex_2 = []

    if(pd.notnull(row['clean_reviews'])):
        for term in ht_dict.keys():
            term_padded = " " + re.escape(term) + " "
            row_padded = " " + row['clean_reviews'] + " "
            indices = [(m.start(), m.end()) for m in re.finditer(term_padded, row_padded)]
            if indices != []:
                if ht_dict[term] == 0:
                    ht_0.extend(indices)
                elif ht_dict[term] == 1:
                    ht_1.extend(indices)
                elif ht_dict[term] == 2:
                    ht_2.extend(indices)
                    
                if sex_dict[term] == 0:
                    sex_0.extend(indices)
                elif sex_dict[term] == 1:
                    sex_1.extend(indices)
                elif sex_dict[term] == 2:
                    sex_2.extend(indices)   
                    
    ht_0.sort(key=lambda x: x[0])
    ht_1.sort(key=lambda x: x[0])
    ht_2.sort(key=lambda x: x[0])
    
    sex_0.sort(key=lambda x: x[0])
    sex_1.sort(key=lambda x: x[0])
    sex_2.sort(key=lambda x: x[0])
    
    HT_uni_0_count = len(ht_0)
    HT_uni_1_count = len(ht_1)
    HT_uni_2_count = len(ht_2)
    sex_uni_0_count = len(sex_0)
    sex_uni_1_count = len(sex_1)
    sex_uni_2_count = len(sex_2)
    
    HT_bi_0_count = count_bigrams(ht_0)
    HT_bi_1_count = count_bigrams(ht_1)
    HT_bi_2_count = count_bigrams(ht_2)
    HT_tri_0_count = count_trigrams(ht_0)
    HT_tri_1_count = count_trigrams(ht_2)
    HT_tri_2_count = count_trigrams(ht_2)

    sex_bi_0_count = count_bigrams(sex_0)
    sex_bi_1_count = count_bigrams(sex_1)
    sex_bi_2_count = count_bigrams(sex_2)
    sex_tri_0_count = count_trigrams(sex_0)
    sex_tri_1_count = count_trigrams(sex_1)
    sex_tri_2_count = count_trigrams(sex_2)
    
    row['HT_bi_0_count'] = HT_bi_0_count
    row['HT_bi_1_count'] = HT_bi_1_count
    row['HT_bi_2_count'] = HT_bi_2_count
    row['HT_tri_0_count'] = HT_tri_0_count
    row['HT_tri_1_count'] = HT_tri_1_count
    row['HT_tri_2_count'] = HT_tri_2_count
    row['HT_uni_0_count'] = HT_uni_0_count
    row['HT_uni_1_count'] = HT_uni_1_count
    row['HT_uni_2_count'] = HT_uni_2_count
    
    row['sex_bi_0_count'] = sex_bi_0_count
    row['sex_bi_1_count'] = sex_bi_1_count
    row['sex_bi_2_count'] = sex_bi_2_count
    row['sex_tri_0_count'] = sex_tri_0_count
    row['sex_tri_1_count'] = sex_tri_1_count
    row['sex_tri_2_count'] = sex_tri_2_count
    row['sex_uni_0_count'] = sex_uni_0_count
    row['sex_uni_1_count'] = sex_uni_1_count
    row['sex_uni_2_count'] = sex_uni_2_count
    
    return row

def count_bigrams(l):
    bis = 0
    for i in range(len(l) - 1):
        # if end of current overlaps with start of next
        if l[i][1] >= l[i+1][0]:
            bis += 1
    return bis

def count_trigrams(l):
    tris = 0
    for i in range(len(l) - 2):
        # if end of current overlaps with start of next and end of next 
        # overlaps start of next next
        if l[i][1] >= l[i+1][0] and l[i+1][1] >= l[i+2][0]:
            tris += 1
    return tris

In [66]:
reviews_new = reviews.apply(lex_score_real, axis=1)

In [89]:
reviews2.iloc[1]['clean_reviews']

'attractive looking women involved experience really wanted see sarah tina tiffany also sarah really pretty looking made feel good went experience wanting felt hands move around body hints getting session sarah smile looked cute sure dressed conservatively didnt get flip foot massage nice massage awesome felt like sarah great girl bad didnt really area anyway'

Notable things that are not in the lexicon: 
"hand job" ,
"titty" ,
"titty sucking"

In [72]:
reviews2.iloc[3]

HT_bi_0_count                                                              0
HT_bi_1_count                                                              0
HT_bi_2_count                                                              0
HT_total_lexicon_score                                                     0
HT_tri_0_count                                                             0
HT_tri_1_count                                                             0
HT_tri_2_count                                                             0
HT_uni_0_count                                                             5
HT_uni_1_count                                                             0
HT_uni_2_count                                                             0
actual                                                                     1
clean_reviews              changed name sign says asian massage massage 6...
lemm_reviews               change name sign say asian massage massage 6 1...

In [68]:
reviews_new.iloc[3]

HT_bi_0_count                                                              1
HT_bi_1_count                                                              0
HT_bi_2_count                                                              0
HT_total_lexicon_score                                                     0
HT_tri_0_count                                                             0
HT_tri_1_count                                                             0
HT_tri_2_count                                                             0
HT_uni_0_count                                                             4
HT_uni_1_count                                                             0
HT_uni_2_count                                                             0
actual                                                                     1
clean_reviews              changed name sign says asian massage massage 6...
lemm_reviews               change name sign say asian massage massage 6 1...

In [58]:

def lex_score(row):
    HT_total_lexicon_score = 0
    sex_total_lexicon_score = 0
    
    HT_bi_0_count = 0
    HT_bi_1_count = 0
    HT_bi_2_count = 0
    HT_tri_0_count = 0
    HT_tri_1_count = 0
    HT_tri_2_count = 0
    HT_uni_0_count = 0
    HT_uni_1_count = 0
    HT_uni_2_count = 0
    
    sex_bi_0_count = 0
    sex_bi_1_count = 0
    sex_bi_2_count = 0
    sex_tri_0_count = 0
    sex_tri_1_count = 0
    sex_tri_2_count = 0
    sex_uni_0_count = 0
    sex_uni_1_count = 0
    sex_uni_2_count = 0
    
    bi_count = 0
    tri_count = 0
    
    sex_bi_score = 0
    sex_tri_score = 0
    HT_bi_score = 0
    HT_tri_score = 0
    
    prev_any_terms = 0

    prev_sex_0_terms = 0
    prev_sex_1_terms = 0
    prev_sex_2_terms = 0
    
    prev_HT_0_terms = 0
    prev_HT_1_terms = 0
    prev_HT_2_terms = 0
    
    if(pd.notnull(row['clean_reviews'])):
        for word_upper in row['clean_reviews'].split(' '):
            word=word_upper.lower().strip(" ")
            print(word)
            if word in ht_dict.keys():
                HT_total_lexicon_score += ht_dict[word]
                sex_total_lexicon_score += sex_dict[word]
                
                if prev_any_terms == 1:
                    bi_count += 1
                    HT_bi_score += ht_dict[word]
                    sex_bi_score += sex_dict[word]
                    
                elif prev_any_terms > 1:
                    tri_count += 1
                    HT_tri_score += ht_dict[word]
                    sex_tri_score += sex_dict[word]
                    
                prev_any_terms += 1
                
                if ht_dict[word] == 0:
                    HT_uni_0_count += 1
                    
                    if prev_HT_0_terms == 1:
                        HT_bi_0_count += 1
                    elif prev_HT_0_terms > 1:
                        HT_tri_0_count += 1
                    
                    prev_HT_0_terms += 1
                    prev_HT_1_terms = 0
                    prev_HT_2_terms = 0
                    
                elif ht_dict[word] == 1:
                    HT_uni_1_count += 1
                    
                    if prev_HT_1_terms == 1:
                        HT_bi_1_count += 1
                    elif prev_HT_1_terms > 1:
                        HT_tri_1_count += 1
                        
                    prev_HT_1_terms += 1
                    prev_HT_0_terms = 0
                    prev_HT_2_terms = 0
                    
                elif ht_dict[word] == 2:
                    HT_uni_2_count += 1
                    
                    if prev_HT_2_terms == 1:
                        HT_bi_2_count += 1
                    elif prev_HT_2_terms > 1:
                        HT_tri_2_count += 1
                        
                    prev_HT_2_terms += 1
                    prev_HT_0_terms = 0
                    prev_HT_1_terms = 0
                    
                if sex_dict[word] == 0:
                    sex_uni_0_count += 1
                    
                    if prev_sex_0_terms == 1:
                        sex_bi_0_count += 1
                    elif prev_sex_0_terms > 1:
                        sex_tri_0_count += 1
                    
                    prev_sex_0_terms += 1
                    prev_sex_1_terms = 0
                    prev_sex_2_terms = 0
                    
                elif sex_dict[word] == 1:
                    sex_uni_1_count += 1
                    
                    if prev_sex_1_terms == 1:
                        sex_bi_1_count += 1
                    elif prev_sex_1_terms > 1:
                        sex_tri_1_count += 1
                    
                    prev_sex_0_terms = 0
                    prev_sex_1_terms += 1
                    prev_sex_2_terms = 0
                    
                elif sex_dict[word] == 2:
                    sex_uni_2_count += 1
                    
                    if prev_sex_2_terms == 1:
                        sex_bi_2_count += 1
                    elif prev_sex_2_terms > 1:
                        sex_tri_2_count += 1
                    
                    prev_sex_0_terms = 0
                    prev_sex_1_terms = 0
                    prev_sex_2_terms += 1
            else:
                prev_sex_0_terms = 0
                prev_sex_1_terms = 0
                prev_sex_2_terms = 0
                prev_any_terms = 0

                prev_HT_0_terms = 0
                prev_HT_1_terms = 0
                prev_HT_2_terms = 0
    
    row['HT_total_lexicon_score'] = HT_total_lexicon_score
    row['sex_total_lexicon_score'] = sex_total_lexicon_score
    
    row['HT_bi_0_count'] = HT_bi_0_count
    row['HT_bi_1_count'] = HT_bi_1_count
    row['HT_bi_2_count'] = HT_bi_2_count
    row['HT_tri_0_count'] = HT_tri_0_count
    row['HT_tri_1_count'] = HT_tri_1_count
    row['HT_tri_2_count'] = HT_tri_2_count
    row['HT_uni_0_count'] = HT_uni_0_count
    row['HT_uni_1_count'] = HT_uni_1_count
    row['HT_uni_2_count'] = HT_uni_2_count
    
    row['sex_bi_0_count'] = sex_bi_0_count
    row['sex_bi_1_count'] = sex_bi_1_count
    row['sex_bi_2_count'] = sex_bi_2_count
    row['sex_tri_0_count'] = sex_tri_0_count
    row['sex_tri_1_count'] = sex_tri_1_count
    row['sex_tri_2_count'] = sex_tri_2_count
    row['sex_uni_0_count'] = sex_uni_0_count
    row['sex_uni_1_count'] = sex_uni_1_count
    row['sex_uni_2_count'] = sex_uni_2_count
    
    row['bi_count'] = bi_count
    row['tri_count'] = tri_count
    
    row['sex_bi_score'] = sex_bi_score
    row['sex_tri_score'] = sex_tri_score
    row['HT_bi_score'] = HT_bi_score
    row['HT_tri_score'] = HT_tri_score
    return row

reviews = reviews.apply(lex_score, axis=1)
#reviews.to_pickle('review_features.p')
#runtime = time.time() - start_time
#with open("features_runtime.txt", 'w') as file:
    #file.write(str(runtime))


In [32]:
ht_0_ours = [(117, 122), (127, 140), (246, 259), (455, 459), (121, 128), (458, 467), (528, 538)]
for l,u in ht_0_ours:
    print(l,u)
    print("'" + reviews_new.iloc[2]['clean_reviews'][l:u] + "'")

117 122
'cmt l'
127 140
'deep tissue a'
246 259
'deep tissue b'
455 459
'dt m'
121 128
'level d'
458 467
'massage r'
528 538
'provider g'


In [44]:
ht_0_ours.sort(key=lambda x: x[0])

In [50]:
ht_0_ours

[(117, 122),
 (121, 128),
 (127, 140),
 (246, 259),
 (455, 459),
 (458, 467),
 (528, 538)]

In [53]:
def count_bigrams(l):
    bis = 0
    for i in range(len(l) - 1):
        # if end of current overlaps with start of next
        if l[i][1] >= l[i+1][0]:
            bis += 1
    return bis

def count_trigrams(l):
    tris = 0
    for i in range(len(l) - 2):
        # if end of current overlaps with start of next and end of next 
        # overlaps start of next next
        if l[i][1] >= l[i+1][0] and l[i+1][1] >= l[i+2][0]:
            tris += 1
    return tris

In [54]:
count_bigrams(ht_0_ours)

3

In [55]:
count_trigrams(ht_0_ours)

1